## Notebook setup

In [ ]:
import os
import io
import warnings
import datetime
import numpy as np
import base64

from IPython.display import HTML

# import sys
# sys.path.append('../../')

from openbb_terminal.reports import widget_helpers as widgets
from openbb_terminal.sdk import openbb
from openbb_terminal.core.config.paths import REPOSITORY_DIRECTORY
from openbb_terminal import OpenBBFigure, TerminalStyle

warnings.filterwarnings("ignore")

# Detect if prediction capabilities are present. If they are not, disable prediction in the rest of the script
# so that the report can still be generated without prediction results.
# predictions = True
# try:
#     openbb.stocks.pred.models
# except Exception as e:
#     predictions = False

# TODO Fix predictions virtual path on api refactored

predictions = False

In [ ]:
try:
    theme = TerminalStyle("light", "light")
except:
    pass
stylesheet = widgets.html_report_stylesheet()

## Select Ticker

In [ ]:
# Parameters that will be replaced when calling this notebook
# Do not leave parameters blank as notebook will not run otherwise
symbol = "VTI"
report_name = "Equity Report for VTI"

In [ ]:
ticker = symbol

In [ ]:
if "." in ticker:
    import sys

    sys.exit(0)

In [ ]:
ticker_data = openbb.stocks.load(
    symbol=ticker, start_date=datetime.datetime.now() - datetime.timedelta(days=4 * 30)
)
ticker_data = openbb.stocks.process_candle(data=ticker_data)

author = "OpenBB"
report_title = f"ETF RESEARCH REPORT ON {ticker.upper()}"
report_date = datetime.datetime.now().strftime("%d %B, %Y")
report_time = datetime.datetime.now().strftime("%H:%M")
report_timezone = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
report_title, report_date, report_time, report_timezone

In [ ]:
# info = openbb.etf.summary(name=ticker)
# info

## Data

In [ ]:
data = openbb.stocks.load(ticker)

candle_chart = openbb.etf.candle(
    symbol=ticker,
    data=data,
    external_axes=True,
)

if candle_chart:
    candle_chart = candle_chart.to_html()

In [ ]:
import pandas as pd

sectors = openbb.etf.weights(ticker)
sector_weights_formatted = {}
for sector_weight in sectors:
    sector_weights_formatted[sector_weight["sector"]] = (
        float(sector_weight["weightPercentage"].strip("%")) / 100
    )
sector_weights_formatted = dict(sorted(sector_weights_formatted.items()))
weights_df = pd.DataFrame.from_dict(sector_weights_formatted, orient="index")
weights_df.columns = ["Percentage"]

In [ ]:
weights_chart = ""

if len(weights_df[weights_df["Percentage"] > 10].index) > 1:
    colors = theme.get_colors()
    fig = OpenBBFigure.create_subplots(
        1,
        3,
        specs=[[{"type": "domain"}, {"type": "pie", "colspan": 2}, None]],
        row_heights=[1],
        column_widths=[0.1, 0.8, 0.1],
    )

    fig.add_pie(
        labels=weights_df[weights_df["Percentage"] > 10].index,
        values=weights_df[weights_df["Percentage"] > 10]["Percentage"],
        textinfo="label+percent",
        hoverinfo="label+percent",
        automargin=True,
        rotation=45,
        row=1,
        col=2,
    )
    fig.update_traces(
        textposition="outside",
        textfont_size=15,
        marker=dict(
            colors=colors,
            line=dict(color="#F5EFF3", width=0.8),
        ),
    )

    fig.update_layout(
        margin=dict(t=40, b=20),
        title=dict(
            text=f"Sector holdings of {ticker}",
            y=0.98,
            x=0.5,
            xanchor="center",
            yanchor="top",
        ),
        colorway=colors,
        showlegend=False,
    )

    weights_chart = fig.to_html()

In [ ]:
holdings = openbb.etf.holdings(ticker)
holdings

## Render the report template to a file

In [ ]:
body = f"""
<style>
@font-face {{
    font-family: "Fira Code";
    src: url("{(OpenBBFigure.plotlyjs_path.parent.parent / "web"/"css").as_uri()}/fonts/FiraCode-Regular.ttf") format("truetype");
    font-weight: 400 700;
    font-stretch: 50%;
}}
</style>
<script src="/{OpenBBFigure.plotlyjs_path.as_uri()}"></script>
"""

img = (
    str(REPOSITORY_DIRECTORY)
    + "/openbb_terminal/reports/templates/OpenBB_reports_logo.png"
)
floppy_disk_img = (
    str(REPOSITORY_DIRECTORY) + "/openbb_terminal/reports/templates/floppy-disc.png"
)
body += widgets.header(
    img,
    floppy_disk_img,
    author,
    report_date,
    report_time,
    report_timezone,
    f"<b>ETF RESEARCH REPORT:</b> {ticker.upper()}",
)

body += widgets.tablinks(["SUMMARY"])

htmlcode = widgets.h(3, "Summary")
# htmlcode += widgets.p(info)
htmlcode += widgets.row([candle_chart])

htmlcode += widgets.p("The ETF has the following weights distribution:")

htmlcode += widgets.row([weights_df.to_html()])
htmlcode += widgets.row([weights_chart])

htmlcode += widgets.p("And has funds in the following assets:")
htmlcode += widgets.row([holdings.to_html()])

body += widgets.add_tab("SUMMARY", htmlcode)

body += widgets.tab_clickable_and_save_evt()

report = widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)

# to save the results
with open(report_name + ".html", "w", encoding="utf-8") as fh:
    fh.write(report)